# 610 FINAL - Kaggle Competition Submit## Nicolas Horn - Workflow Gerencial Optimizado---### Resumen de Experimentos Realizados| Version | Tecnicas | Ganancia Promedio | Resultado ||---------|----------|-------------------|----------|| v2 | Base: lags + deltas + trends (3,6) | 6,456,000 | Baseline || v4 | v2 + Rolling Stats + Random Forest | 6,267,840 | Empeoro (-3%) || v5 | v2 + Canaritos (feature selection) | ~6,200,000 | Empeoro || **v6** | **v2 + Hiperparametros BO optimizados** | **En prueba** | **Esta version** |### Conclusiones de Experimentos1. **Rolling Statistics y RF no mejoraron**: Agregar mas features (media movil, desv. estandar, variables de Random Forest) aumento el ruido sin mejorar la ganancia.2. **Canaritos no funciono bien**: La seleccion de features con canaritos elimino variables que en realidad aportaban.3. **Los trends son clave**: Las pendientes de regresion (trend_3, trend_6) capturan la tendencia del comportamiento del cliente y son muy predictivas.4. **Hiperparametros importan**: La Bayesian Optimization encontro que un arbol mas complejo (212 hojas) con menos iteraciones (729) funciona mejor.---### Tecnicas Finales Implementadas1. **Catastrophe Analysis**: 13 variables con datos anomalos en junio 2020 -> NA2. **Data Drifting**: Ajuste por IPC para hacer comparables los valores monetarios3. **Feature Engineering Historico**:   - Lags orden 1 y 2   - Deltas (diferencias con lags)   - Trends ventana 3 y 6 meses (pendiente de regresion lineal)4. **Hiperparametros optimizados por BO**:   - num_leaves = 212   - min_data_in_leaf = 1000 (ajustado de 2 para evitar overfitting)   - num_iterations = 729---**Competencia:** labo-i-2025-virtual-gerencial**5 semillas:** 153929, 838969, 922081, 795581, 194609

---## 1. Configuracion del Ambiente (Google Colab)Esta seccion configura el entorno de Google Colab:- Monta Google Drive para persistencia de archivos- Configura credenciales de Kaggle para submit automatico- Descarga el dataset de la competencia**Runtime: Python 3** (solo para esta celda de montaje)

In [ ]:
# Montar Google Drive para tener persistencia de archivos entre sesionesfrom google.colab import drivedrive.mount('/content/.drive')

In [ ]:
%%shell# =============================================================================# CONFIGURACION DE DIRECTORIOS Y DESCARGA DE DATOS# =============================================================================mkdir -p "/content/.drive/My Drive/labo1"mkdir -p "/content/buckets"ln -sf "/content/.drive/My Drive/labo1" /content/buckets/b1# Configurar Kaggle APImkdir -p ~/.kaggleif [ -f "/content/buckets/b1/kaggle/kaggle.json" ]; then  cp /content/buckets/b1/kaggle/kaggle.json ~/.kaggle/  chmod 600 ~/.kaggle/kaggle.json  echo "OK: Kaggle configurado"else  echo "ADVERTENCIA: kaggle.json no encontrado"fimkdir -p /content/buckets/b1/expmkdir -p /content/buckets/b1/datasetsmkdir -p /content/datasets# Descargar datasetwebfiles="https://storage.googleapis.com/open-courses/austral2025-af91"archivo="gerencial_competencia_2025.csv.gz"if [ ! -f "/content/buckets/b1/datasets/$archivo" ]; then  wget -q "$webfiles/$archivo" -O "/content/buckets/b1/datasets/$archivo"fiif [ ! -f "/content/datasets/$archivo" ]; then  cp "/content/buckets/b1/datasets/$archivo" "/content/datasets/$archivo"fils -lh "/content/datasets/$archivo" 

---## 2. Inicializacion de R**IMPORTANTE: Cambiar Runtime a R** antes de continuar.Menu -> Runtime -> Change Runtime Type -> R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# Limpieza de memoriarm(list=ls(all.names=TRUE))gc(full=TRUE, verbose=FALSE)

In [ ]:
# Carga de libreriasrequire("data.table")if(!require("R.utils")) install.packages("R.utils")require("R.utils")if(!require("lightgbm")) install.packages("lightgbm")require("lightgbm")if(!require("yaml")) install.packages("yaml")require("yaml")

---## 3. Parametros GlobalesConfiguracion central del experimento:- **Semillas**: 5 semillas para tener resultados reproducibles y medir variabilidad- **Competencia Kaggle**: Nombre de la competencia para submit automatico- **Cortes de envio**: Diferentes cantidades de clientes a enviar para encontrar el optimo

In [ ]:
# =============================================================================# PARAMETROS GLOBALES# =============================================================================PARAM_GLOBAL <- list()# Identificador del experimento (6170 = v6 + Kaggle)PARAM_GLOBAL$experimento_base <- 6170# Dataset de la competencia gerencial# Contiene 15 meses: 202005 a 202107 (para experimentar) + 202109 (para predecir)PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"# 5 semillas para reproducibilidad y medir varianzaPARAM_GLOBAL$semillas <- c(153929, 838969, 922081, 795581, 194609)# Configuracion de KagglePARAM_GLOBAL$kaggle_competencia <- "labo-i-2025-virtual-gerencial"# Cortes de envio: probar diferentes cantidades de clientesPARAM_GLOBAL$kaggle_cortes <- seq(800, 1400, by = 50)# Lista para almacenar resultadosresultados_totales <- list()cat("Experimento base:", PARAM_GLOBAL$experimento_base, "\n")cat("Semillas:", PARAM_GLOBAL$semillas, "\n")cat("Cortes Kaggle:", PARAM_GLOBAL$kaggle_cortes, "\n")

---## 4. Indices para Correccion de Data Drifting (IPC)### Problema: Data Drifting MonetarioLos valores monetarios no son comparables entre meses debido a la inflacion.Un saldo de $10,000 en mayo 2020 no es lo mismo que $10,000 en julio 2021.### Solucion: Deflactacion por IPCMultiplicamos los valores monetarios por un indice que los lleva a pesos constantes de diciembre 2020 (momento 1.0).- Meses anteriores a dic-2020: IPC > 1 ("inflar" al valor de dic-2020)- Meses posteriores a dic-2020: IPC < 1 ("desinflar" al valor de dic-2020)Esto hace que los valores sean comparables en el tiempo.

In [ ]:
# =============================================================================# INDICES IPC PARA DEFLACTACION# =============================================================================# Valores calculados por alumnos del curso# Momento 1.0 = 31-dic-2020 a las 23:59vfoto_mes <- c(  202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,  202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108, 202109)vIPC <- c(  1.2118694724, 1.1881073259,  # 202005, 202006  1.1693969743, 1.1375456949, 1.1065619600,  # 202007-202009  1.0681100000, 1.0370000000, 1.0000000000,  # 202010-202012 (dic=1.0)  0.9680542110, 0.9344152616, 0.8882274350,  # 202101-202103  0.8532444140, 0.8251880213, 0.8003763543,  # 202104-202106  0.7763107219, 0.7566381305, 0.7289384687   # 202107-202109)tb_indices <- data.table(foto_mes = vfoto_mes, IPC = vIPC)print(tb_indices)

---## 5. Funcion para Calcular Tendencia (Trend)### Por que usar Trends?Los trends capturan la **direccion** del cambio en el comportamiento del cliente:- Trend positivo: el cliente esta aumentando su actividad- Trend negativo: el cliente esta disminuyendo su actividad (posible baja)### ImplementacionCalculamos la **pendiente de regresion lineal** sobre una ventana de tiempo.Usamos formula analitica (sin `lm()`) para mayor velocidad.

In [ ]:
# =============================================================================# FUNCION PARA CALCULAR PENDIENTE (TREND)# =============================================================================# Calcula la pendiente de regresion lineal de forma optimizada# Maneja NAs correctamente: solo usa los valores validos disponiblescalc_slope_fast <- function(y) {  n <- length(y)  valid <- !is.na(y)  n_valid <- sum(valid)  if (n_valid < 2) return(NA_real_)  x <- 1:n  x_valid <- x[valid]  y_valid <- y[valid]  sum_x <- sum(x_valid)  sum_y <- sum(y_valid)  sum_xy <- sum(x_valid * y_valid)  sum_x2 <- sum(x_valid^2)  denom <- n_valid * sum_x2 - sum_x^2  if (denom == 0) return(NA_real_)  (n_valid * sum_xy - sum_x * sum_y) / denom}# Testcat("Test calc_slope_fast:\n")cat("  Pendiente de c(1,2,3):", calc_slope_fast(c(1,2,3)), "(esperado: 1)\n")cat("  Pendiente de c(3,2,1):", calc_slope_fast(c(3,2,1)), "(esperado: -1)\n")

---## 6. Loop Principal - 5 Semillas con Submit a KaggleEste loop ejecuta el pipeline completo para cada semilla:1. **Carga de datos**: Lee el dataset comprimido2. **Catastrophe Analysis**: Marca como NA las variables con datos anomalos3. **Data Drifting**: Ajusta valores monetarios por IPC4. **Feature Engineering**: Crea lags, deltas y trends5. **Entrenamiento**: LightGBM con hiperparametros optimizados6. **Prediccion**: Genera probabilidades para el mes 2021097. **Submit a Kaggle**: Envia multiples cortes automaticamente### Hiperparametros (de Bayesian Optimization)- `num_leaves = 212`: Arbol mas complejo que el default- `min_data_in_leaf = 1000`: Ajustado de 2 para evitar overfitting- `num_iterations = 729`: Menos iteraciones, evita sobreajuste- `feature_fraction = 0.5`: Usa 50% de features por arbol- `learning_rate = 0.03`: Aprendizaje lento para mejor generalizacion

In [ ]:
# =============================================================================# LOOP PRINCIPAL - 5 SEMILLAS CON SUBMIT A KAGGLE# =============================================================================for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {  cat("\n\n")  cat("##########################################################\n")  cat("# SEMILLA", seed_idx, "de", length(PARAM_GLOBAL$semillas), "\n")  cat("# Valor:", PARAM_GLOBAL$semillas[seed_idx], "\n")  cat("##########################################################\n\n")  inicio_seed <- Sys.time()  # =========================================================================  # INICIALIZACION DE PARAMETROS PARA ESTA SEMILLA  # =========================================================================  PARAM <- list()  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1  PARAM$dataset <- PARAM_GLOBAL$dataset  # Crear carpeta del experimento  setwd("/content/buckets/b1/exp")  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx, "_FINAL_kaggle")  dir.create(experimento_folder, showWarnings=FALSE)  setwd(paste0("/content/buckets/b1/exp/", experimento_folder))  dir.create("kaggle", showWarnings=FALSE)  cat("Carpeta:", experimento_folder, "\n\n")  # =========================================================================  # 6.1 CARGA DEL DATASET  # =========================================================================  cat("[1/7] Cargando dataset...\n")  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))  cat("      Dimensiones:", nrow(dataset), "filas x", ncol(dataset), "columnas\n\n")  # =========================================================================  # 6.2 CATASTROPHE ANALYSIS  # =========================================================================  # En junio 2020 hubo problemas de datos en 13 variables  # La solucion de Machine Learning es marcarlos como NA  # LightGBM maneja NAs nativamente de forma optima  cat("[2/7] Catastrophe Analysis (13 variables en 202006 -> NA)...\n")  dataset[foto_mes==202006, internet := NA]  dataset[foto_mes==202006, mrentabilidad := NA]  dataset[foto_mes==202006, mrentabilidad_annual := NA]  dataset[foto_mes==202006, mcomisiones := NA]  dataset[foto_mes==202006, mactivos_margen := NA]  dataset[foto_mes==202006, mpasivos_margen := NA]  dataset[foto_mes==202006, mcuentas_saldo := NA]  dataset[foto_mes==202006, ctarjeta_visa_transacciones := NA]  dataset[foto_mes==202006, mtarjeta_visa_consumo := NA]  dataset[foto_mes==202006, mtarjeta_master_consumo := NA]  dataset[foto_mes==202006, ccallcenter_transacciones := NA]  dataset[foto_mes==202006, chomebanking_transacciones := NA]  dataset[foto_mes==202006, ctarjeta_master_transacciones := NA]  cat("      Completado\n\n")  # =========================================================================  # 6.3 DATA DRIFTING - CORRECCION POR IPC  # =========================================================================  cat("[3/7] Data Drifting (ajuste por IPC)...\n")  campos_monetarios <- colnames(dataset)[colnames(dataset) %like% "^m"]  cat("      Variables monetarias:", length(campos_monetarios), "\n")  dataset[tb_indices,          on = "foto_mes",          (campos_monetarios) := .SD * i.IPC,          .SDcols = campos_monetarios]  cat("      Completado\n\n")  # =========================================================================  # 6.4 FEATURE ENGINEERING INTRA-MES  # =========================================================================  cat("[4/7] Feature Engineering...\n")  cat("      FE Intra-mes...\n")  dataset[, kmes := foto_mes %% 100]  if("mpayroll" %in% colnames(dataset) & "cliente_edad" %in% colnames(dataset)) {    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]  }  # =========================================================================  # 6.5 FEATURE ENGINEERING HISTORICO  # =========================================================================  cat("      FE Historico (lags + deltas + trends)...\n")  inicio_fe <- Sys.time()  setorder(dataset, numero_de_cliente, foto_mes)  cols_lagueables <- setdiff(colnames(dataset),                             c("numero_de_cliente", "foto_mes", "clase_ternaria"))  # --- LAGS ---  dataset[, paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),          by = numero_de_cliente, .SDcols = cols_lagueables]  dataset[, paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),          by = numero_de_cliente, .SDcols = cols_lagueables]  # --- DELTAS ---  for (vcol in cols_lagueables) {    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]  }  # --- TRENDS ---  for (col in cols_lagueables) {    dataset[, paste0(col, "_trend_3") := frollapply(      get(col), 3, calc_slope_fast, align="right"    ), by = numero_de_cliente]  }  for (col in cols_lagueables) {    dataset[, paste0(col, "_trend_6") := frollapply(      get(col), 6, calc_slope_fast, align="right"    ), by = numero_de_cliente]  }  tiempo_fe <- round(difftime(Sys.time(), inicio_fe, units="mins"), 1)  cat("      FE completado en", tiempo_fe, "min\n")  cat("      Dataset final:", ncol(dataset), "columnas\n\n")  # =========================================================================  # 6.6 TRAINING STRATEGY  # =========================================================================  cat("[5/7] Configurando entrenamiento...\n")  PARAM$trainingstrategy <- list()  PARAM$trainingstrategy$final_train <- c(    202107, 202106, 202105, 202104, 202103, 202102, 202101,    202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005  )  PARAM$trainingstrategy$future <- c(202109)  dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1, 0)]  campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01", "azar"))  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]  cat("      Meses de entrenamiento:", length(PARAM$trainingstrategy$final_train), "\n")  cat("      Features:", length(campos_buenos), "\n\n")  # =========================================================================  # 6.7 ENTRENAMIENTO LIGHTGBM  # =========================================================================  cat("[6/7] Entrenando modelo LightGBM...\n")  cat("      Hiperparametros:\n")  cat("        num_leaves = 212 (de BO)\n")  cat("        min_data_in_leaf = 1000 (ajustado)\n")  cat("        num_iterations = 729 (de BO)\n")  cat("        feature_fraction = 0.5\n")  cat("        learning_rate = 0.03\n")  dfinal_train <- lgb.Dataset(    data = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),    label = dataset[fold_final_train == TRUE, clase01],    free_raw_data = TRUE  )  param_final <- list(    objective = "binary",    metric = "auc",    first_metric_only = TRUE,    boost_from_average = TRUE,    feature_pre_filter = FALSE,    verbosity = -100,    force_row_wise = TRUE,    seed = PARAM$semilla_primigenia,    max_bin = 31,    # Hiperparametros de BO + ajustes    num_leaves = 212,    min_data_in_leaf = 1000,    num_iterations = 729,    feature_fraction = 0.5,    learning_rate = 0.03  )  inicio_train <- Sys.time()  final_model <- lgb.train(data = dfinal_train, param = param_final, verbose = -100)  tiempo_train <- round(difftime(Sys.time(), inicio_train, units="mins"), 1)  cat("      Modelo entrenado en", tiempo_train, "min\n\n")  lgb.save(final_model, "modelo.txt")  fwrite(as.data.table(lgb.importance(final_model)), file = "impo.txt", sep = "\t")  # =========================================================================  # 6.8 PREDICCION Y SUBMIT A KAGGLE  # =========================================================================  cat("[7/7] Scoring y submit a Kaggle...\n")  dfuture <- dataset[foto_mes %in% PARAM$trainingstrategy$future]  prediccion <- predict(final_model, data.matrix(dfuture[, campos_buenos, with = FALSE]))  tb_prediccion <- dfuture[, list(numero_de_cliente)]  tb_prediccion[, prob := prediccion]  fwrite(tb_prediccion, file = "prediccion.txt", sep = "\t")  setorder(tb_prediccion, -prob)  cat("      Enviando", length(PARAM_GLOBAL$kaggle_cortes), "submits a Kaggle...\n")  for (envios in PARAM_GLOBAL$kaggle_cortes) {    tb_prediccion[, Predicted := 0L]    tb_prediccion[1:envios, Predicted := 1L]    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")    fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],           file = archivo_kaggle,           sep = ",")    mensaje_kaggle <- paste0("seed_", PARAM$semilla_primigenia, "_corte_", envios)    cmd <- paste0("kaggle competitions submit -c ",                  PARAM_GLOBAL$kaggle_competencia,                  " -f ", archivo_kaggle,                  " -m '", mensaje_kaggle, "'")    tryCatch({      system(cmd, intern = TRUE)      cat("      -> Enviado corte", envios, "\n")    }, error = function(e) {      cat("      -> Error en corte", envios, "\n")    })    Sys.sleep(30)  # Pausa entre submits para no saturar Kaggle  }  # Guardar parametros usados  write_yaml(PARAM, "parametros.yml")  tiempo_seed <- round(difftime(Sys.time(), inicio_seed, units="mins"), 1)  cat("\n      Semilla", seed_idx, "completada en", tiempo_seed, "min\n")  # Guardar resultado  resultados_totales[[seed_idx]] <- list(    semilla = PARAM$semilla_primigenia,    experimento = PARAM$experimento,    tiempo_min = tiempo_seed  )  # Limpiar memoria antes de la siguiente semilla  rm(dataset, dfinal_train, final_model, dfuture, tb_prediccion)  gc(full=TRUE, verbose=FALSE)}cat("\n\n")cat("##########################################################\n")cat("# PROCESO COMPLETADO\n")cat("##########################################################\n")cat("\nResumen:\n")for (i in 1:length(resultados_totales)) {  cat("  Semilla", i, ":", resultados_totales[[i]]$semilla,      "- Exp:", resultados_totales[[i]]$experimento,      "- Tiempo:", resultados_totales[[i]]$tiempo_min, "min\n")}

In [ ]:
# Timestamp finalformat(Sys.time(), "%a %b %d %X %Y")